In [1]:
import numpy as np
import json
import random
random.seed(12781278)

In [2]:
n_nodes = 200
n_edges_per_node = 10
minNodeSize = 5
maxNodeSize = 50
n_groups = 5
n_contrasts = 3
n_statistics = 3
contrast_names = ["ctrl_vs_treat_" + str(iter) for iter in range(1, n_contrasts+1)]
statisticsMeasuresUnivariate = ["statMeasure_" + str(iter) for iter in range(1, n_statistics+1)]
statisticsMeasuresGroupBased = ["globalTest"]
assert n_nodes % n_groups == 0, "Require n_nodes to be a multiple of n_groups"

all_possible_corners = []
for xiter in range(0, 1500, 500):
  for yiter in range(0, 1000, 500):
    all_possible_corners.append([xiter, yiter])

assert len(all_possible_corners) >= n_groups, "Require more possible cluster centers than groups!"
group_left_corners = random.sample(all_possible_corners, n_groups)

Beware: for some configurations of this dev tool the while loop may get stuck because not enough edges exist for the configuration sought!

In [3]:
nodes = []
edges = []
groups = []
nodes_per_group = int(n_nodes / n_groups) # because automatically set to float...
group_ids = ["group"+str(iter) for iter in range(0, n_groups)]
for iloc, group_id in enumerate(group_ids):
  # generate group specific data
  node_ids = ["node"+str(iter)+str(group_id) for iter in range(0, nodes_per_group)]
  group_corner_x = group_left_corners[iloc][0]
  group_corner_y = group_left_corners[iloc][1]
  for node_id in node_ids:
    datatmp = {contrastKey:{measureKey: {"measure":random.uniform(0,1),"nodeSize": random.randint(minNodeSize, maxNodeSize)} for measureKey in statisticsMeasuresUnivariate} for contrastKey in contrast_names}
    nodes.append(
      {
        "id": node_id, 
        "size": random.randint(minNodeSize, maxNodeSize),
        "group": group_id,
        "x": group_corner_x + random.randint(0, 600),
        "y": group_corner_y + random.randint(0, 600),
        "data": datatmp
      }
    )
  used_node_combinations = set()
  edge_count = 0
  for iloc, node in enumerate(node_ids):
    for iter in range(0, n_edges_per_node):
      current_combination = [node, random.choice(node_ids)]
      while frozenset(current_combination) in used_node_combinations or current_combination[0] == current_combination[1]:
        current_combination = [node, random.choice(node_ids)]
      used_node_combinations.add(frozenset(current_combination))
      edges.append({
          "id": str(group_id) + "edge" + str(edge_count), 
          "from": current_combination[0], 
          "to": current_combination[1],
          "width": random.randint(1, 30),
          "data": {"score": random.uniform(0,1)}
        }
      )
      edge_count = edge_count + 1 
   

In [4]:
nodes[1]


{'id': 'node1group0',
 'size': 31,
 'group': 'group0',
 'x': 1062,
 'y': 794,
 'data': {'ctrl_vs_treat_1': {'statMeasure_1': {'measure': 0.8410657210177551,
    'nodeSize': 50},
   'statMeasure_2': {'measure': 0.13373237410207528, 'nodeSize': 45},
   'statMeasure_3': {'measure': 0.06138049180282146, 'nodeSize': 12}},
  'ctrl_vs_treat_2': {'statMeasure_1': {'measure': 0.2544714201322119,
    'nodeSize': 28},
   'statMeasure_2': {'measure': 0.5278156216378298, 'nodeSize': 14},
   'statMeasure_3': {'measure': 0.9732333392937361, 'nodeSize': 26}},
  'ctrl_vs_treat_3': {'statMeasure_1': {'measure': 0.820107333020695,
    'nodeSize': 22},
   'statMeasure_2': {'measure': 0.7545333527980925, 'nodeSize': 36},
   'statMeasure_3': {'measure': 0.27128904035926393, 'nodeSize': 43}}}}

In [5]:

group_stats = {
  groupKey: {
    contrastKey: {
      measureKey: random.uniform(0,1) 
      for measureKey in statisticsMeasuresGroupBased} 
    for contrastKey in contrast_names} 
  for groupKey in group_ids
}


group_stats

{'group0': {'ctrl_vs_treat_1': {'globalTest': 0.2745125736092313},
  'ctrl_vs_treat_2': {'globalTest': 0.29743761758405685},
  'ctrl_vs_treat_3': {'globalTest': 0.5621221329937915}},
 'group1': {'ctrl_vs_treat_1': {'globalTest': 0.33854336495737447},
  'ctrl_vs_treat_2': {'globalTest': 0.9446481158456699},
  'ctrl_vs_treat_3': {'globalTest': 0.5379245883758641}},
 'group2': {'ctrl_vs_treat_1': {'globalTest': 0.3775799645174156},
  'ctrl_vs_treat_2': {'globalTest': 0.013712010680358544},
  'ctrl_vs_treat_3': {'globalTest': 0.5134425280522228}},
 'group3': {'ctrl_vs_treat_1': {'globalTest': 0.0352377657859656},
  'ctrl_vs_treat_2': {'globalTest': 0.09478408534424942},
  'ctrl_vs_treat_3': {'globalTest': 0.9760450495978709}},
 'group4': {'ctrl_vs_treat_1': {'globalTest': 0.1763537055843869},
  'ctrl_vs_treat_2': {'globalTest': 0.26086761531254277},
  'ctrl_vs_treat_3': {'globalTest': 0.22099984110465498}}}

In [6]:
with open("mockNodesEdgesDemo-v5-large.json", "w") as file:
    json.dump(
        {
            "groupKeys": group_ids,
            "univMeasureKeys": statisticsMeasuresUnivariate, 
            "groupMeasureKeys": statisticsMeasuresGroupBased,
            "contrastKeys": contrast_names,
            "groupStats": group_stats, 
            "nodes": nodes, 
            "edges": edges
        },
        file, indent=2
    )